In [1]:
!pip install --upgrade google-colab
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.1 MB/s eta 0:00:00
Mounted at /content/drive


In [2]:
!git clone https://github.com/WongKinYiu/yolov7.git

Cloning into 'yolov7'...
remote: Enumerating objects: 1197, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 1197 (delta 2), reused 3 (delta 1), pack-reused 1191
Receiving objects: 100% (1197/1197), 74.24 MiB | 24.36 MiB/s, done.
Resolving deltas: 100% (518/518), done.


In [3]:
%cd yolov7

!pip install -r requirements.txt

/content/yolov7


In [4]:
#Grab labels for COCO dataset

import requests
import os
import zipfile
import json
from tqdm import tqdm

# URL for the COCO dataset annotations
annotations_url = "http://images.cocodataset.org/annotations/annotations_trainval2017.zip"

# Function to download a file given a URL
def download_file(url, filename):
    response = requests.get(url, stream=True)
    total_size_in_bytes = int(response.headers.get('content-length', 0))
    block_size = 1024  # 1 Kibibyte
    progress_bar = tqdm(total=total_size_in_bytes, unit='iB', unit_scale=True)
    with open(filename, 'wb') as file:
        for data in response.iter_content(block_size):
            progress_bar.update(len(data))
            file.write(data)
    progress_bar.close()
    if total_size_in_bytes != 0 and progress_bar.n != total_size_in_bytes:
        print("ERROR, something went wrong")

# Download annotations
annotations_filename = "coco_annotations.zip"
download_file(annotations_url, annotations_filename)

# Extract annotations
with zipfile.ZipFile(annotations_filename, 'r') as zip_ref:
    zip_ref.extractall("coco_annotations")

# Load the class labels
labels_filename = os.path.join("coco_annotations", "annotations", "instances_val2017.json")
with open(labels_filename, 'r') as f:
    data = json.load(f)
    categories = data['categories']

# Extract category names
class_labels = {category['id']: category['name'] for category in categories}
print(class_labels)

# Optionally, save the class labels to a text file
with open('coco_class_labels.txt', 'w') as f:
    for category_id, category_name in class_labels.items():
        f.write(f"{category_id}: {category_name}\n")


100%|██████████| 253M/253M [00:13<00:00, 18.3MiB/s]


{1: 'person', 2: 'bicycle', 3: 'car', 4: 'motorcycle', 5: 'airplane', 6: 'bus', 7: 'train', 8: 'truck', 9: 'boat', 10: 'traffic light', 11: 'fire hydrant', 13: 'stop sign', 14: 'parking meter', 15: 'bench', 16: 'bird', 17: 'cat', 18: 'dog', 19: 'horse', 20: 'sheep', 21: 'cow', 22: 'elephant', 23: 'bear', 24: 'zebra', 25: 'giraffe', 27: 'backpack', 28: 'umbrella', 31: 'handbag', 32: 'tie', 33: 'suitcase', 34: 'frisbee', 35: 'skis', 36: 'snowboard', 37: 'sports ball', 38: 'kite', 39: 'baseball bat', 40: 'baseball glove', 41: 'skateboard', 42: 'surfboard', 43: 'tennis racket', 44: 'bottle', 46: 'wine glass', 47: 'cup', 48: 'fork', 49: 'knife', 50: 'spoon', 51: 'bowl', 52: 'banana', 53: 'apple', 54: 'sandwich', 55: 'orange', 56: 'broccoli', 57: 'carrot', 58: 'hot dog', 59: 'pizza', 60: 'donut', 61: 'cake', 62: 'chair', 63: 'couch', 64: 'potted plant', 65: 'bed', 67: 'dining table', 70: 'toilet', 72: 'tv', 73: 'laptop', 74: 'mouse', 75: 'remote', 76: 'keyboard', 77: 'cell phone', 78: 'micro

In [8]:
#Define useful Functions
import cv2
import torch
import torch.nn.functional as F

def preprocess_frame(frame, height, width):
    """
    Preprocess the frame for model input.

    Parameters:
    frame (ndarray): The frame to be processed.
    height (int): The height to resize the frame to.
    width (int): The width to resize the frame to.

    Returns:
    torch.Tensor: The preprocessed frame as a tensor.
    """
    # Resize the frame to the target size
    frame_resized = cv2.resize(frame, (width, height))
    # Normalize the frame by dividing by 255
    frame_normalized = frame_resized / 255.0
    # Get the device from the model parameters
    device = next(yolov7_tiny_model.parameters()).device
    # Convert the frame to a PyTorch tensor and add a batch dimension
    frame_tensor = torch.from_numpy(frame_normalized).float().permute(2, 0, 1).unsqueeze(0)
    # Move the tensor to the device and convert to half precision
    frame_tensor = frame_tensor.to(device).half()
    return frame_tensor

import torch
from torchvision.ops import nms

def process_results(results, confidence_threshold=0.5):
    """
    Process the raw results from the model.

    Parameters:
    results (torch.Tensor): The raw results from the model.
    confidence_threshold (float): The threshold for considering detections.

    Returns:
    Tuple: A tuple containing confidence scores, bounding boxes, and class predictions.
    """
    # Apply sigmoid to the last dimension (class predictions)
    predictions = torch.sigmoid(results[..., 5:])
    # Apply sigmoid to the objectness scores
    confidence_scores = torch.sigmoid(results[..., 4])
    # Apply sigmoid to the bounding box coordinates
    boxes = torch.sigmoid(results[..., :4])

    # Filter out the results below the confidence threshold
    mask = confidence_scores > confidence_threshold
    filtered_boxes = boxes[mask]
    filtered_scores = confidence_scores[mask]
    filtered_predictions = predictions[mask]

    # Here we need to determine the class with the maximum prediction for each filtered box
    class_predictions = torch.argmax(filtered_predictions, dim=-1)

    # Convert to lists
    boxes_list = filtered_boxes.tolist()
    scores_list = filtered_scores.tolist()
    class_predictions_list = class_predictions.tolist()

    return scores_list, boxes_list, class_predictions_list



def draw_boxes_on_frame(frame, boxes, original_image_width, original_image_height, model_name):
    """
    Draw bounding boxes on the frame.

    Parameters:
    frame (ndarray): The frame to draw the boxes on.
    boxes (list): The list of bounding boxes.
    original_image_width (int): The original width of the frame.
    original_image_height (int): The original height of the frame.
    model_name (str): The name of the model to display on the frame.
    """
    # Draw each box on the frame
    for box in boxes:
        # Scale the box coordinates back to the original frame size
        x_center, y_center, width, height = box
        x_center *= original_image_width
        y_center *= original_image_height
        width *= original_image_width
        height *= original_image_height
        # Convert from center coordinates to top-left coordinates
        x = int(x_center - width / 2)
        y = int(y_center - height / 2)
        # Draw the rectangle on the frame
        cv2.rectangle(frame, (x, y), (x + int(width), y + int(height)), (0, 255, 0), 2)
    # Put the model name text on the frame
    cv2.putText(frame, model_name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

def process_predictions(predictions, class_subset, threshold):
    """
    Process predictions and execute an action if any predictions in a subset exceed a threshold.

    Parameters:
    predictions (list): List of predictions for all classes.
    class_subset (list): List of indices for the subset of classes to check.
    prediction_indices (list): List of prediction indices to check within the subset.
    threshold (float): The threshold above which a prediction is considered positive.
    action (function): A function to execute if a positive prediction is found.
    """
    # Check if any specified prediction index is in the subset and exceeds the threshold
    for class_index in predictions:
        if class_index in class_subset:
            use_large = True
            return
    use_large = False
    return use_large

def alert():
    use_large = True
    print("Positive prediction found!")

def evaluate_predictions(predicted_indices, ground_truth_indices):
    """
    Compare given prediction indices for a frame against the ground truth indices.

    Parameters:
    predicted_indices (list): The list of predicted class indices for the current frame.
    ground_truth_indices (list): The list of ground truth class indices for the current frame.

    Returns:
    Tuple: Counts of true positives, false positives, and false negatives.
    """
    TP = FP = FN = 0
    # Convert ground truth and predictions to sets for efficient comparison
    predicted_indices_set = set(predicted_indices)
    ground_truth_indices_set = set(ground_truth_indices)
    '''
    print(predicted_indices_set)
    print(ground_truth_indices_set)
    print(frame_count)
    '''
    # True Positives (TP): Predicted indices that are in the ground truth
    TP = len(predicted_indices_set.intersection(ground_truth_indices_set))

    # False Positives (FP): Predicted indices that are not in the ground truth
    FP = len(predicted_indices_set.difference(ground_truth_indices_set))

    # False Negatives (FN): Ground truth indices that were not predicted
    FN = len(ground_truth_indices_set.difference(predicted_indices_set))

    return TP, FP, FN

def calculate_evaluation_metrics(TP, FP, FN):
    """
    Calculate evaluation metrics based on the counts of true positives, false positives, true negatives, and false negatives.

    Parameters:
    TP (int): Number of true positives.
    FP (int): Number of false positives.
    TN (int): Number of true negatives.
    FN (int): Number of false negatives.

    Returns:
    Dict: A dictionary with TPR, Precision, and Recall.
    """
    metrics = {
        'TPR': TP / (TP + FN) if (TP + FN) > 0 else 0,
        'Precision': TP / (TP + FP) if (TP + FP) > 0 else 0,
    }
    return metrics

import torch
from torchvision.ops import box_iou, nms

def calculate_average_agreement(boxes1, boxes2, predictions1, predictions2, iou_threshold=0.5):
    """
    Calculates the average agreement between two sets of object detection predictions.

    Parameters:
    - boxes1 (List[List[float]]): Bounding boxes from the first model.
    - boxes2 (List[List[float]]): Bounding boxes from the second model.
    - predictions1 (List[int]): Class predictions from the first model.
    - predictions2 (List[int]): Class predictions from the second model.
    - iou_threshold (float): Threshold above which boxes are considered matching.

    Returns:
    - float: The average agreement score.
    """
    # Ensure that boxes are tensors
    if isinstance(boxes1, list):
        boxes1 = torch.tensor(boxes1, dtype=torch.float32)
    else:
        boxes1 = boxes1.clone().detach()

    if isinstance(boxes2, list):
        boxes2 = torch.tensor(boxes2, dtype=torch.float32)
    else:
        boxes2 = boxes2.clone().detach()

    if boxes1.nelement() == 0 and boxes2.nelement() == 0:
        return 1.0  # No agreement if both sets are empty


    # Ensure that boxes are not empty and are tensors
    # Check if any of the tensors are empty, and if so, return 0.0 as the agreement score
    if boxes1.nelement() == 0 or boxes2.nelement() == 0:
        return 0.0

    boxes1 = torch.tensor(boxes1, dtype=torch.float32)
    boxes2 = torch.tensor(boxes2, dtype=torch.float32)

    # Calculate the IoU for each pair of boxes
    ious = box_iou(boxes1, boxes2)

    # Determine which boxes match based on IoU threshold
    matches = ious > iou_threshold

    total_matches = 0
    for i in range(matches.size(0)):
        for j in range(matches.size(1)):
            if matches[i, j] and predictions1[i] == predictions2[j]:
                total_matches += 1

    # Calculate the agreement score
    total_predictions = max(len(predictions1), len(predictions2))
    agreement_score = total_matches / total_predictions if total_predictions > 0 else 0

    return agreement_score



import csv
import time

def write_to_csv(data, filename):
    with open(filename, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=['frame_count', 'average_agreement_score'])
        writer.writeheader()
        for frame_data in data:
            writer.writerow(frame_data)

def process_video(interval, Skip_Frames=False, Context_Sensitive=False):
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Define the codec
    cap = cv2.VideoCapture('file path to .mp4 video') #This is were you change the video input make sure you match the following size
    out = cv2.VideoWriter('output_video.mp4', fourcc, 20.0, (1280, 720))
    TP = FP = FN = 0
    frame_count = 0
    total_time = 0
    use_large = False
    threshold = 0.5

    while cap.isOpened():
        start_time = time.time()
        ret, frame = cap.read()
        frame_count += 1
        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break
        if frame_count % 5 != 0 and Skip_Frames:
            continue

        # Use the larger model every 10 frames
        if frame_count % interval == 0:
          use_large = True

            # Perform inference using the appropriate model
        if use_large:
            # Preprocess the current frame to prepare for model input
            frame_tensor = preprocess_frame(frame, 640, 640)
            results = yolov7_model(frame_tensor)
            results = results[0]
            model_name = "yolov7"
        else:
            # Preprocess the current frame to prepare for model input
            frame_tensor = preprocess_frame(frame, 608, 608)
            results = yolov7_tiny_model(frame_tensor)
            results = results[0]
            model_name = "yolov7-tiny"

        # Check if results are in a list and select the first element
        if isinstance(results, list):
            results = results[0]

        # Process the results to get confidence scores, bounding boxes, and predictions
        confidence_scores, boxes, predictions = process_results(results, threshold)

        # Draw bounding boxes on the frame
        draw_boxes_on_frame(frame, boxes, original_image_width, original_image_height, model_name)

        # Assuming class_labels is defined elsewhere and contains class indices as keys
        class_indices = list(class_labels.keys())  # Get all class indices

        # Exclude the first 10 classes for processing
        subset_indices = class_indices[10:]

        if Context_Sensitive:
          # Process predictions to check for any detections in the subset of classes
          process_predictions(predictions, subset_indices, threshold)
        else:
          use_large = False

        # Write the processed frame to the output video
        out.write(frame)


        frame_predictions_indices = frame_predictions[str(frame_count)]

        # Draw bounding boxes on the frame
        draw_boxes_on_frame(frame, boxes, original_image_width, original_image_height, model_name)

        # Evaluate the predictions against the ground truth for the current frame
        frame_TP, frame_FP, frame_FN = evaluate_predictions(predictions, frame_predictions_indices)

        # Update the overall confusion matrix counters
        TP += frame_TP
        FP += frame_FP
        FN += frame_FN

        # Write the processed frame to the output video
        out.write(frame)


        # Calculate processing time and update total time
        end_time = time.time()
        total_time += (end_time - start_time)


    # Release video resources
    cap.release()
    out.release()
    print(frame_count)
    print(total_time)
    # Calculate FPS and evaluation metrics
    fps = frame_count / total_time
    evaluation_metrics = calculate_evaluation_metrics(TP, FP, FN)
    return evaluation_metrics, fps





In [18]:
#Run the Model Evaluator on "large_model_intervals" and all combinations of "schemes"
#Outputs a CSV file with the results

import cv2
import torch
import sys
import matplotlib.pyplot as plt
import torch.nn.functional as F
import json

large_model_intervals = [10, 20, 40]
schemes = [[False, False], [True, False], [False, True], [True, True]]
# Load pre-trained models
yolov7_model = torch.load('yolov7-e6e .pt file path')['model']
yolov7_tiny_model = torch.load('yolov7-tiny .pt file path')['model']
# Original dimensions of the video frames
original_image_height = 720
original_image_width = 1280

# Load the ground truth predictions from a JSON file
with open('file path to .json updated e6e predicitons', 'r') as f:
    frame_predictions = json.load(f)

# Set the models to evaluation mode
yolov7_model.eval()
yolov7_tiny_model.eval()

# Set the threshold for detection
threshold = 0.5  # Change this value to adjust sensitivity
# Process video for each interval and write results to CSV
with open('evaluation_metrics.csv', 'w', newline='') as csvfile:
    fieldnames = ['Context_Sense', 'Skip_Frame', 'Interval', 'TPR', 'Precision', 'FPS']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for scheme in schemes:
        Skip_Frames = scheme[0]
        Context_Sense = scheme[1]
        for interval in large_model_intervals:
            metrics, fps = process_video(interval, Skip_Frames, Context_Sense)
            writer.writerow({'Context_Sense': f"{Context_Sense}", 'Skip_Frame': f"{Skip_Frames}", 'Interval': f"{interval}", 'TPR': metrics['TPR'], 'Precision': metrics['Precision'], 'FPS': fps})

print("Processing complete. Metrics written to evaluation_metrics.csv")

Can't receive frame (stream end?). Exiting ...
5596
528.2009534835815
Can't receive frame (stream end?). Exiting ...
5596
510.1961884498596
Can't receive frame (stream end?). Exiting ...
5596
504.02336716651917
Can't receive frame (stream end?). Exiting ...
5596
113.66189193725586
Can't receive frame (stream end?). Exiting ...
5596
107.37122964859009
Can't receive frame (stream end?). Exiting ...
5596
103.84329557418823
Can't receive frame (stream end?). Exiting ...
5596
612.0269241333008
Can't receive frame (stream end?). Exiting ...
5596
612.6276412010193
Can't receive frame (stream end?). Exiting ...
5596
611.8702168464661
Can't receive frame (stream end?). Exiting ...
5596
124.1579692363739
Can't receive frame (stream end?). Exiting ...
5596
124.21319603919983
Can't receive frame (stream end?). Exiting ...
5596
124.02974963188171
Processing complete. Metrics written to evaluation_metrics.csv


In [ ]:
#To do a single run of the model evaluator
import cv2
import torch
import sys
import matplotlib.pyplot as plt
import torch.nn.functional as F
import json

large_model_intervals = [10, 20, 40]
schemes = [[False, False], [True, False], [False, True], [True, True]]
# Load pre-trained models
yolov7_model = torch.load('yolov7-e6e .pt file path')['model']
yolov7_tiny_model = torch.load('yolov7-tiny .pt file path')['model']
# Original dimensions of the video frames
original_image_height = 720
original_image_width = 1280

# Load the ground truth predictions from a JSON file
with open('file path to .json updated e6e predicitons', 'r') as f:
    frame_predictions = json.load(f)

# Set the models to evaluation mode
yolov7_model.eval()
yolov7_tiny_model.eval()

# Set the threshold for detection
threshold = 0.5  # Change this value to adjust sensitiviy

metrics, fps = process_video(10, False, False)

print(f"TPR: {metrics['TPR']}")
print(f"Precision: {metrics['Precision']}")
print(f"FPS: {fps}")



Can't receive frame (stream end?). Exiting ...
5596
503.30618810653687
TPR: 0.7985921655428012
Precision: 0.4894625589576136
FPS: 11.118480424515408


In [16]:
#Run for a single instance without outside function
import cv2
import torch
import sys
import matplotlib.pyplot as plt
import torch.nn.functional as F
import json

# Load pre-trained models
yolov7_model = torch.load('yolov7-e6e .pt file path')['model']
yolov7_tiny_model = torch.load('yolov7-tiny .pt file path')['model']


# Set the models to evaluation mode
yolov7_model.eval()
# Set the threshold for detection
threshold = 0.5  # Change this value to adjust sensitivity

# Initialize frame count and flags
frame_count = 0
use_large = False


# Original dimensions of the video frames
original_image_height = 720
original_image_width = 1280

# Initialize confusion matrix counters
TP = FP = FN = 0


# Load the ground truth predictions from a JSON file
with open('file path to .json updated e6e predicitons', 'r') as f:
    frame_predictions = json.load(f)

# Open the video file
cap = cv2.VideoCapture('file path to .mp4 video')

# Set up the VideoWriter object to save the output video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Define the codec
out = cv2.VideoWriter('output_video.mp4', fourcc, 20.0, (1280, 720))  # Set the output file name, codec, fps, and resolution

# Process video frames
while cap.isOpened():
    ret, frame = cap.read()
    frame_count += 1
    if not ret:
        print("Failed to grab a frame")
        break  # Exit the loop if no frame is grabbed
    if frame_count % 5 != 0:
        continue



    # Use the larger model every 10 frames
    if frame_count % 40 == 0:
        use_large = True



    # Perform inference using the appropriate model
    if use_large:
        # Preprocess the current frame to prepare for model input
        frame_tensor = preprocess_frame(frame, 640, 640)
        results = yolov7_model(frame_tensor)
        results = results[0]
        model_name = "yolov7"
    else:
        # Preprocess the current frame to prepare for model input
        frame_tensor = preprocess_frame(frame, 608, 608)
        results = yolov7_tiny_model(frame_tensor)
        model_name = "yolov7-tiny"

    # Check if results are in a list and select the first element
    if isinstance(results, list):
        results = results[0]

    # Process the results to get confidence scores, bounding boxes, and predictions
    confidence_scores, boxes, predictions = process_results(results, threshold)

    # Draw bounding boxes on the frame
    draw_boxes_on_frame(frame, boxes, original_image_width, original_image_height, model_name)

    # Assuming class_labels is defined elsewhere and contains class indices as keys
    class_indices = list(class_labels.keys())  # Get all class indices

    # Exclude the first 10 classes for processing
    subset_indices = class_indices[10:]

    # Process predictions to check for any detections in the subset of classes
    process_predictions(predictions, subset_indices, threshold)


    frame_predictions_indices = frame_predictions[str(frame_count)]

    # Draw bounding boxes on the frame
    draw_boxes_on_frame(frame, boxes, original_image_width, original_image_height, model_name)

    # Evaluate the predictions against the ground truth for the current frame
    frame_TP, frame_FP, frame_FN = evaluate_predictions(predictions, frame_predictions_indices)

    # Update the overall confusion matrix counters
    TP += frame_TP
    FP += frame_FP
    FN += frame_FN

    # Write the processed frame to the output video
    out.write(frame)

# Release video resources
cap.release()
out.release()


# Calculate and print evaluation metrics
evaluation_metrics = calculate_evaluation_metrics(TP, FP, FN)
print(f"True Positive Rate (TPR): {evaluation_metrics['TPR']}")
print(f"Precision: {evaluation_metrics['Precision']}")
print("Final Frame Count:", frame_count)



Failed to grab a frame
True Positive Rate (TPR): 1.0
Precision: 1.0
Final Frame Count: 5596


In [15]:
#producing prediction dictionary
import cv2
import torch
import sys
import matplotlib.pyplot as plt
import torch.nn.functional as F
import json

# Load pre-trained models
yolov7_model = torch.load('yolov7-e6e .pt file path')['model']
yolov7_tiny_model = torch.load('yolov7-tiny .pt file path')['model']

frame_predictions_dict = {}

# Load the class labels
# Set the models to evaluation mode
yolov7_model.eval()
# Set the threshold for detection
threshold = 0.5  # Change this value to adjust sensitivity

# Initialize frame count and flags
frame_count = 0
use_large = False


# Original dimensions of the video frames
original_image_height = 720
original_image_width = 1280

# Initialize confusion matrix counters
TP = FP = FN = 0


# Load the ground truth predictions from a JSON file
with open('file path to .json updated e6e predicitons', 'r') as f:
    frame_predictions = json.load(f)

# Open the video file
cap = cv2.VideoCapture('file path to .mp4 video')

# Set up the VideoWriter object to save the output video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Define the codec
out = cv2.VideoWriter('output_video.mp4', fourcc, 20.0, (1280, 720))  # Set the output file name, codec, fps, and resolution

# Process video frames
while cap.isOpened():
    ret, frame = cap.read()
    frame_count += 1
    if not ret:
        print("Failed to grab a frame")
        break  # Exit the loop if no frame is grabbed
    if frame_count % 5 != 0 and False:
        continue



    # Use the larger model every 10 frames
    if frame_count % 40 == 0:
        use_large = True



    # Perform inference using the appropriate model
    if use_large or True:
        # Preprocess the current frame to prepare for model input
        frame_tensor = preprocess_frame(frame, 640, 640)
        results = yolov7_model(frame_tensor)
        results = results[0]
        model_name = "yolov7"
    else:
        # Preprocess the current frame to prepare for model input
        frame_tensor = preprocess_frame(frame, 608, 608)
        results = yolov7_tiny_model(frame_tensor)
        model_name = "yolov7-tiny"

    # Check if results are in a list and select the first element
    if isinstance(results, list):
        results = results[0]

    # Process the results to get confidence scores, bounding boxes, and predictions
    confidence_scores, boxes, predictions = process_results(results, threshold)

    # Draw bounding boxes on the frame
    draw_boxes_on_frame(frame, boxes, original_image_width, original_image_height, model_name)

    # Assuming class_labels is defined elsewhere and contains class indices as keys
    class_indices = list(class_labels.keys())  # Get all class indices

    # Exclude the first 10 classes for processing
    subset_indices = class_indices[10:]

    # Process predictions to check for any detections in the subset of classes
    process_predictions(predictions, subset_indices, threshold)


    frame_predictions_indices = frame_predictions[str(frame_count)]

    # Draw bounding boxes on the frame
    draw_boxes_on_frame(frame, boxes, original_image_width, original_image_height, model_name)

    frame_predictions_dict[str(frame_count)] = predictions

    # Evaluate the predictions against the ground truth for the current frame
    frame_TP, frame_FP, frame_FN = evaluate_predictions(predictions, frame_predictions_indices)

    # Update the overall confusion matrix counters
    TP += frame_TP
    FP += frame_FP
    FN += frame_FN

    # Write the processed frame to the output video
    out.write(frame)

# Release video resources
cap.release()
out.release()

with open('/content/drive/MyDrive/EdgeAI/Collab Folder/BigModelOutput/frame_predictions_update_e6e.json', 'w') as outfile:
    json.dump(frame_predictions_dict, outfile)

# Calculate and print evaluation metrics
evaluation_metrics = calculate_evaluation_metrics(TP, FP, FN)
print(f"True Positive Rate (TPR): {evaluation_metrics['TPR']}")
print(f"Precision: {evaluation_metrics['Precision']}")
print("Final Frame Count:", frame_count)

Failed to grab a frame
True Positive Rate (TPR): 0.7785016286644951
Precision: 0.027680547691357388
Final Frame Count: 5596


In [ ]:
#Capturing model agreement on frame bases
import cv2
import torch
import sys
import matplotlib.pyplot as plt
import torch.nn.functional as F
import json

# Load pre-trained models
yolov7_model = torch.load('yolov7-e6e .pt file path')['model']
yolov7_tiny_model = torch.load('yolov7-tiny .pt file path')['model']

# Set the threshold for detection
threshold = 0.5  # Change this value to adjust sensitivity

# Initialize frame count and flags
frame_count = 0

# Initialize a list to hold the data dictionaries
data_to_write = []

# Original dimensions of the video frames
original_image_height = 720
original_image_width = 1280

# Initialize confusion matrix counters
TP = FP = TN = FN = 0

# Load the ground truth predictions from a JSON file
with open('file path to .json updated e6e predicitons', 'r') as f:
    frame_predictions = json.load(f)

# Open the video file
cap = cv2.VideoCapture('file path to .mp4 video')

# Set up the VideoWriter object to save the output video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Define the codec
out = cv2.VideoWriter('output_video.mp4', fourcc, 20.0, (1280, 720))  # Set the output file name, codec, fps, and resolution

# Process video frames
while cap.isOpened():
    ret, frame = cap.read()
    frame_count += 1
    if not ret:
        print("Failed to grab a frame")
        break  # Exit the loop if no frame is grabbed



    # Preprocess the current frame to prepare for model input
    frame_tensor = preprocess_frame(frame, 608, 608)

    results_large = yolov7_model(frame_tensor)
    results_tiny = yolov7_tiny_model(frame_tensor)

    # Check if results are in a list and select the first element
    if isinstance(results_large, list):
        results_large = results_large[0]
    if isinstance(results_tiny, list):
        results_tiny = results_tiny[0]
    confidence_boxes, boxes_tiny, predictions_tiny = process_results(results_tiny, 0.5)
    confidence_boxes, boxes_large, predictions_large = process_results(results_large, 0.5)

    average_agreement = calculate_average_agreement(boxes_tiny, boxes_large, predictions_tiny, predictions_large)

    data_to_write.append({
        'frame_count': frame_count,
        'average_agreement_score': average_agreement
    })

    # Evaluate the predictions against the ground truth for the current frame
    frame_TP, frame_FP, frame_TN, frame_FN = evaluate_predictions(frame_count, predictions_large, frame_predictions, threshold)

    if frame_count == 1000:
        write_to_csv(data_to_write, "Model_Agreement_Object_Detection.csv")
        print("Data written to CSV file")
        break

    # Draw bounding boxes on the frame



    # Draw bounding boxes and predictions on the frame

<ipython-input-26-3ad473cb1f45>:202: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  boxes1 = torch.tensor(boxes1, dtype=torch.float32)
<ipython-input-26-3ad473cb1f45>:203: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  boxes2 = torch.tensor(boxes2, dtype=torch.float32)


Data written to CSV file
